# Ortogonalize canonical basis on simplex

In [27]:
import sys
sys.path.append("/home/tomas/Python_projects/sfepy")

In [28]:
from dg_basis import iter_by_order

In [29]:
from toolz import reduce as redc
from operator import mul

In [30]:
def gramm_schmidt(base, dot_prod):
    '''
    Ortogonalizuje bázi base vůči skálárnímu součinu dot_prod
    :param base:
    :param dot_prod:
    :return:
    '''
    new_base = []
    for i in range(len(base)):
        new_base.append(ortogonalize(base[i], new_base, dot_prod))
    return new_base


def ortogonalize(vec, base, dot_prod):
    for j in range(len(base)):
        kap = - dot_prod(vec, base[j]) / dot_prod(base[j], base[j])
        vec = vec + kap * base[j]
    return vec

In [31]:
var("x", "y", "z")
variables = [x, y, z]

order = 2
D = 2

In [32]:
simplexP = []
for m, idx in enumerate(iter_by_order(order, D)):
    tmpf = redc(mul, [variables[d]**i for d, i in enumerate(idx)])
    simplexP.append(tmpf)
    print("P_{} = {}".format(m, tmpf))

P_0 = 1
P_1 = x
P_2 = x^2
P_3 = y
P_4 = x*y
P_5 = y^2


In [33]:
simplex_dots = [lambda f, g: integrate(f*g, (x, 0, 1)),
                lambda f, g: integrate(
                    integrate(f*g, (y, 0, 1 - x)),(x, 0, 1)),
                lambda f, g: integrate(
                    integrate(
                        integrate(f*g, (z, 0, y)),(y, 0, 1 - x)),(x, 0, 1))
               ]

In [47]:
simplexP_ort = gramm_schmidt(simplexP, simplex_dots[D - 1])
simplexP_ort

[1,
 x - 1/3,
 x^2 - 4/5*x + 1/10,
 1/2*x + y - 1/2,
 1/2*x^2 + x*y - 3/5*x - 1/5*y + 1/10,
 1/6*x^2 + x*y + y^2 - 1/3*x - y + 1/6]

In [48]:
Np = len(simplexP_ort)
from sympy.matrices import ones
vendM = ones(Np, Np)
for i in range(Np):
    for j in range(Np):
        vendM[i, j] = simplex_dots[D - 1](simplexP_ort[i],
                                        simplexP_ort[j])# Fubini, yayy!
vendM

Matrix([
[1/2,    0,     0,    0,      0,      0],
[  0, 1/36,     0,    0,      0,      0],
[  0,    0, 1/600,    0,      0,      0],
[  0,    0,     0, 1/48,      0,      0],
[  0,    0,     0,    0, 1/1800,      0],
[  0,    0,     0,    0,      0, 1/1080]])